## Get Train Test Split

In [7]:
import os
import shutil
import cv2
import sys
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import albumentations as A

print("Torch:", torch.__version__)
print("MPS built:", torch.backends.mps.is_built())
print("MPS available:", torch.backends.mps.is_available())

from PIL import Image
from tqdm import tqdm

import numpy as np
# import matplotlib as plt

import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms

# Download latest version
# plant_disease_path = kagglehub.dataset_download("emmarex/plantdisease")
# plantify_dr_path = kagglehub.dataset_download("lavaman151/plantifydr-dataset")

#!ls -l '/content/drive/MyDrive/ECS_170/data/raw'
#!ls -l '/content/drive/Shareddrives/ECS_170/Tomato_Dataset_Splitted_V1.zip'

# unzip datasets
#!unzip '/content/drive/Shareddrives/ECS_170/Tomato_Dataset_Splitted_V1.zip' -d '/content/drive/Shareddrives/ECS_170'
#!unzip '/content/Shareddrive/MyDrive/ECS_170/data/raw/PlantVillage.zip' -d '/content/drive/MyDrive/ECS_170/data/raw'
# !unzip '/content/drive/MyDrive/ECS_170/data/raw/PlantDiseasesDataset.zip' -d '/content/drive/MyDrive/ECS_170/data/raw'

Torch: 2.10.0.dev20251121
MPS built: True
MPS available: True


## Split the data into three folders(train, test and val)

In [ ]:
#Needs work
#referenced - https://medium.com/@myringoleMLGOD/simple-convolutional-neural-network-cnn-for-dummies-in-pytorch-a-step-by-step-guide-6f4109f6df80
#	          - https://medium.com/@kritiikaaa/convolution-neural-networks-guide-for-your-first-cnn-project-7ea56f7f6960
#           - https://www.geeksforgeeks.org/deep-learning/how-to-handle-overfitting-in-pytorch-models-using-early-stopping/ 
# Formal dataset definition

# !ls -l '/content/drive/MyDrive/ECS_170/data/processed/merged_data'
import os
import shutil
import cv2
import sys
from pathlib import Path
import torch
print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())
print("Torch version:", torch.__version__)
print("Torch location:", torch.__file__)
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
from tqdm import tqdmfads

import numpy as np

import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
#Add thing

#CNN definition
class plantCNN(nn.Module):
	def __init__(self):
		super(plantCNN, self).__init__()
		self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
		self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
		self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

		self.pool = nn.MaxPool2d(2, 2)
		self.relu = nn.ReLU()
		self.flatten = nn.Flatten()

		self.fc1 = nn.Linear(64 * 16 * 16, 128)
		self.fc2 = nn.Linear(128, 10) #Healthy, Bacterial spot, Early blight, Late blight, Leaf Mold, Mosaic virus, Septoria leaf spot, Spider mites, Target Spot, Yellow Leaf Curl Virus

	def forward(self, x):
		x = self.pool(self.relu(self.conv1(x)))
		x = self.pool(self.relu(self.conv2(x)))
		x = self.pool(self.relu(self.conv3(x)))
		x = self.flatten(x)
		x = self.relu(self.fc1(x))
		x = self.fc2(x)
		return x

# Load the dataset
data_root = "Tomato_Dataset_Splitted_RGB/"

train_transform = transforms.Compose([
                                # Adding image arugmenation to the input data
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomVerticalFlip(p=0.5),
                                transforms.RandomRotation(30),
                                # Adding color jitter properties
                                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                                # Applying guassianblur
                                transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
                                transforms.ToTensor()
                            ])

val_transform = transforms.Compose([
                                transforms.ToTensor()
                            ])

train_data = ImageFolder(os.path.join(data_root, "train"), transform=train_transform)
val_data = ImageFolder(os.path.join(data_root, "val"), transform=val_transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

# Setting the maximum amount to epochs to detect early stopping
epochs = 50
# Path to store the Model
modelPath = "models/plant_health_cnn.pth"

# Set up training
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
model = plantCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to implement the early stopping in the algorithm
class earlyStopping():
  def __init__(self, patience=5, delta=0.0):
    self.delta = delta
    self.patience = patience
    self.counter = 0
    self.best_loss = None
    self.best_model_state = None 
    self.early_stop = False
  
  def __call__(self, val_loss, model):
    score = -val_loss
    if self.best_loss is None:
      self.best_loss = score
      self.best_model_state = model.state_dict()
    elif score < self.best_loss + self.delta:
      self.counter += 1
      if self.counter >= self.patience:
        self.early_stop = True
      else:
        self.best_loss = score
        self.best_model_state = model.state_dict()
        self.counter = 0

  def load_best_model(self, model):
    model.load_state_dict(self.best_model_state)

# Training and validation loops
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, labels in tqdm(loader, desc="Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()
    acc = 100 * correct / total
    return running_loss / len(loader), acc

def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    acc = 100 * correct / total
    return running_loss / len(loader), acc

early_stopping = earlyStopping()
for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = eval_epoch(model, val_loader, criterion)
    print(f"\nEpoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%\n")
    # Checking for early stopping
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
      os.makedirs("models/", exist_ok=True)
      torch.save(model.state_dict(), modelPath)
      print("Model Early Stopped")
      break
    elif (epoch+1) == epochs:
      os.makedirs("models/", exist_ok=True)
      torch.save(model.state_dict(), modelPath)
      print("Model Trained")


MPS available: True
MPS built: True
Torch version: 2.10.0.dev20251121
Torch location: /Users/devrathod/Documents/classes_2025/ECS_170/torch_env/lib/python3.11/site-packages/torch/__init__.py


Validation: 100%|██████████| 136/136 [00:29<00:00,  4.61it/s]



Epoch [1/50]
Train Loss: 1.5985 | Train Acc: 43.84%
Val Loss:   1.6290 | Val Acc:   45.04%



Validation: 100%|██████████| 136/136 [00:34<00:00,  3.98it/s]



Epoch [2/50]
Train Loss: 1.1282 | Train Acc: 61.10%
Val Loss:   0.9888 | Val Acc:   65.72%



...


Epoch [48/50]
Train Loss: 0.2850 | Train Acc: 90.08%
Val Loss:   0.6166 | Val Acc:   80.93%



Validation: 100%|██████████| 136/136 [00:19<00:00,  6.95it/s]



Epoch [49/50]
Train Loss: 0.2843 | Train Acc: 90.00%
Val Loss:   0.4739 | Val Acc:   84.87%



Validation: 100%|██████████| 136/136 [00:19<00:00,  6.87it/s]



Epoch [50/50]
Train Loss: 0.2872 | Train Acc: 89.98%
Val Loss:   0.4807 | Val Acc:   84.41%

Model Trained
